In [1]:
import os
import tensorflow as tf
from tensorflow import keras
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "3"
import numpy as np
import json
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix   #copy

In [2]:
data = np.load('lp.npz')       #mark final.npz
label = data['y']
Bigdata = data['X']
Bigdata[np.isnan(Bigdata)] = 0.
# Bigdata = Bigdata[:,:,1:]
results = {}
results['acc'] = []
results['auc'] = []
results['confMat'] = []  #add
fold = 0
label[label < 5] = 0
label[label==5] = 1
y = label
index=np.arange(Bigdata.shape[0])
np.random.shuffle(index)
y=y[index]
Bigdata=Bigdata[index]
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=22)

In [3]:
for train, test in kfold.split(np.zeros(len(y)), y):
    print("run fold:{0}".format(fold))
    train_x, train_y = Bigdata[train], y[train]
    test_x, test_y = Bigdata[test], y[test]
    #train_y, test_y = tf.one_hot(train_y, depth=6).numpy(), tf.one_hot(test_y, depth=6).numpy()
    model = tf.keras.Sequential([
    keras.layers.Masking(mask_value=0., input_shape=(500, 7)), 
    keras.layers.Bidirectional(keras.layers.LSTM(256, return_sequences=True), input_shape=(500, 7)),
    keras.layers.Bidirectional(keras.layers.LSTM(128)),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dense(units=1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss=keras.losses.BinaryCrossentropy(from_logits=False), metrics=['accuracy','AUC'])
    TensorBoard_callback = keras.callbacks.TensorBoard('logs_without_timestamp/' + str(fold) +"/" , histogram_freq=1)
    EarlyStopping_callback = keras.callbacks.EarlyStopping(patience=5)
    checkpoint_callback = keras.callbacks.ModelCheckpoint('checkpoint_without_timestamp/fold_' + str(fold) +"/" , save_best_only=True, save_weights_only=True)
    
    model.fit(x=train_x, y=train_y, batch_size=64, epochs=30, validation_split=0.2, callbacks=[TensorBoard_callback, EarlyStopping_callback, checkpoint_callback], verbose=2)

    #1
    y_pred = model.predict(test_x)
    roc = roc_auc_score(test_y, y_pred)
    y_pred[y_pred >= 0.5 ] =1
    y_pred[y_pred < 0.5] = 0
    acc = accuracy_score(test_y, y_pred)
    confMat = confusion_matrix(test_y, y_pred).astype(str)
    #acc = accuracy_score(np.argmax(test_y, axis=-1), np.argmax(y_pred, axis=-1))
    print("In the fold {0}: acc {1}, roc {2}".format(fold, acc, roc))
    results['acc'].append(acc)
    results['auc'].append(roc)
    confMat = [list(A)for A in confMat]
    results['confMat'].append(confMat)
    fold +=1
    #1

with open('auc_acc_without_timestamp.txt','a') as f:
    f.write(json.dumps(results))

run fold:0
Train on 220 samples, validate on 56 samples
Epoch 1/30
220/220 - 28s - loss: 0.4174 - accuracy: 0.8182 - AUC: 0.9265 - val_loss: 0.1898 - val_accuracy: 0.9286 - val_AUC: 0.9646
Epoch 2/30
220/220 - 1s - loss: 0.1280 - accuracy: 0.9455 - AUC: 0.9919 - val_loss: 0.1163 - val_accuracy: 0.9643 - val_AUC: 0.9891
Epoch 3/30
220/220 - 1s - loss: 0.0656 - accuracy: 0.9727 - AUC: 0.9984 - val_loss: 0.0574 - val_accuracy: 0.9643 - val_AUC: 0.9973
Epoch 4/30
220/220 - 1s - loss: 0.0433 - accuracy: 0.9818 - AUC: 0.9991 - val_loss: 0.0438 - val_accuracy: 0.9821 - val_AUC: 1.0000
Epoch 5/30
220/220 - 1s - loss: 0.0263 - accuracy: 0.9909 - AUC: 0.9997 - val_loss: 0.0390 - val_accuracy: 0.9643 - val_AUC: 1.0000
Epoch 6/30
220/220 - 1s - loss: 0.0181 - accuracy: 0.9955 - AUC: 1.0000 - val_loss: 0.0250 - val_accuracy: 1.0000 - val_AUC: 1.0000
Epoch 7/30
220/220 - 1s - loss: 0.0079 - accuracy: 1.0000 - AUC: 1.0000 - val_loss: 0.0192 - val_accuracy: 1.0000 - val_AUC: 1.0000
Epoch 8/30
220/220 